In [114]:
import pandas as pd
import numpy
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_scoreclassification_report

In [31]:
!pip install torch

You should consider upgrading via the '/Users/lokin/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [33]:
import torch.nn as nn

In [52]:
data = pd.read_csv("ready_to_model_data.csv", index_col = 0)

In [53]:
data.shape

(62938, 41)

In [54]:
data.columns

Index(['PROMISE_STATUS_O', 'EXTENSION_ELIGIBLE_Y',
       'F0223_NUMBER_OF_EXTENSIONS', 'lastchannel_Z', 'REM_TM_RAT_OBS',
       'ApplicantFICOScore', 'Appl_Debt', 'F0066_CONTACT_NO_PROMISE_COUNT',
       'F0315_DAYS_SINCE_LAST_PROMISE_1', 'F0065_PROMISES_TAKEN_COUNT',
       'F0068_ATTEMPTED_CONTACT_COUNT', 'F0071_CORRESPONDENCE_COUNT',
       'F0314_TODAY_MINUS_DAYS_LAST_CONTACT',
       'F0320_TODAY_MINUS_DAYS_LAST_MONETARY', 'F0381_DAYS_DELINQUENT_360',
       'BALANCE_PERCENT_LOAN', 'F0146_PAST_DUE_PAYMENTS', 'DSLP',
       'F0223_NUMBER_OF_EXTENSIONS.1', 'PMT_RAT_OBS', 'PaymentAmtLast60Days',
       'PaymentAmtLast90Days', 'PaymentNumLast120Days',
       'PmtNumFullAmtLast60Days', 'D5P_TOT_1', 'D31P_TOT_6', 'DD_AVG_1',
       'BAL_RAT_TRN_6', 'CBTINDIRECTGEN23', 'LOANFUNDINGSCORE', 'LTVEffective',
       'DepreciationRate', 'Appl_Income', 'PTI_BOOKED', 'SalesTax',
       'DOWN_CASH', 'FINANCED_AMOUNT', 'F0089_PROMISE_AMT_1',
       'F0180_CURRENT_INTEREST_RATE', 'BAL_RAT_TRN_6.1

In [55]:
data['DPD_bool'].unique()

array([False,  True])

In [56]:
data['DPD_bool'] = data['DPD_bool'].map({True: 1, False: 0})

In [57]:
y = data['DPD_bool']
X = data.drop('DPD_bool', axis = 1)

In [58]:
Counter(y)[1]/len(y)

0.15296005592805617

In [119]:
Counter(y)[0]/len(y)

0.8470399440719438

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4)

In [60]:
scaler = StandardScaler()
scale_trainData = scaler.fit_transform(X_train)

In [61]:
scale_trainData

array([[-0.32432413, -1.26136932,  0.54962278, ..., -0.28614241,
        -1.04976712,  0.94250699],
       [-0.32432413, -1.26136932,  0.54962278, ...,  1.67366762,
        -0.65724203,  2.35503841],
       [-0.32432413,  0.79278922, -0.71506892, ..., -0.144921  ,
         1.41215022, -0.26823423],
       ...,
       [-0.32432413,  0.79278922, -0.71506892, ...,  2.18828438,
         0.08070513,  2.35503841],
       [-0.32432413, -1.26136932, -0.71506892, ..., -0.05609669,
        -1.74061127, -0.67181464],
       [-0.32432413,  0.79278922, -0.71506892, ..., -0.10215863,
         2.2474436 , -1.07539504]])

In [95]:
class Net(nn.Module):
    
    def __init__(self, input, H, output):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(input, H)
        self.linear2 = nn.Linear(H, output)
        
    def forward(self, x):
        
        x = torch.from_numpy(x).float()
        x = torch.sigmoid(self.linear1(x))
        x = self.linear2(x)
        
        return x
    
clf = Net(40, 20, 1)

print(clf)

# criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(clf.parameters(), lr = 0.1)


Net(
  (linear1): Linear(in_features=40, out_features=20, bias=True)
  (linear2): Linear(in_features=20, out_features=1, bias=True)
)


In [97]:
# X_train = torch.from_numpy(X_train)
y_train1 = torch.from_numpy(y_train.values)

learning_rate = 1e-1
loss_list = []

for i in range(len(y_train)):
    
    y_pred = clf(scale_trainData[i])
    y_pred = torch.reshape(y_pred, (-1, y_pred.shape[0]))
    
    xz = torch.reshape(y_train1[i], (1,))
    
    xz = torch.tensor(xz, dtype = torch.float64)
    y_pred = torch.tensor(y_pred, dtype = torch.float64)
    
    loss = criterion(y_pred[0], xz)
    
    clf.zero_grad()
    loss.requires_grad = True
    loss.backward()
    
    with torch.no_grad():
        for param in clf.parameters():
            if param.grad is not None:
                param -= learning_rate * param.grad

/Users/lokin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/Users/lokin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


In [98]:
scale_testData = scaler.fit_transform(X_test)

In [112]:
y_test1 = torch.from_numpy(y_test.values)

test_predictions = []

for i in range(len(scale_testData)):
    
    y_test_pred = clf(scale_testData[i])
    y_test_pred = torch.reshape(y_test_pred, (-1, y_test_pred.shape[0]))
    
    xz = torch.reshape(y_test1[i], (1,))
    xz = torch.tensor(xz, dtype = torch.long)
    
    if y_test_pred[0][0].item() > 0.50:
        test_predictions.append(1)
    else:
        test_predictions.append(0)


/Users/lokin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


In [116]:
accuracy_score(y_test, test_predictions) * 100

20.062758182395932

In [117]:
precision_score(y_test, test_predictions) * 100

15.02407980736154

In [118]:
recall_score(y_test, test_predictions) * 100

90.58854031630801